In [1]:
import numpy as np
import random

In [2]:
# Solar Energy System class
class SolarES:
    def __init__(self, paramNamelist, initDict):
        self.Name = initDict['name']
        self.Maxstorage = initDict[paramNamelist[0]]
        self.Storageefficiency = initDict[paramNamelist[1]]
        self.Storage = initDict[paramNamelist[2]]
        self.Maxgeneration = initDict[paramNamelist[3]]
        self.Generation = initDict[paramNamelist[4]]
        self.Maxusage = initDict[paramNamelist[5]]
        self.Usage = initDict[paramNamelist[6]]
        self.Transfer = initDict[paramNamelist[7]]
        self.Generationfailurerate = initDict[paramNamelist[8]]
        self.Storagefailurerate = initDict[paramNamelist[9]]
        self.Generationfailuetime = 0
        self.Generationrepairtime = initDict[paramNamelist[12]]
        self.Storagerepairtime = initDict[paramNamelist[13]]
        self.Storagefailuretime = 0
        self.Generationfailure = 1
        self.Storagefailure = 1
        self.Powercut = 0        
        self.Usagepeaktime = (round(random.gauss(36000, 10800)))%86400 # The energy peak's distribution mean at 10 am and sd of 3 hours
        self.UsageSD = initDict[paramNamelist[10]]
        self.Generationpeaktime = 33200 # The energy peak's distribution mean at 10 am and sd of 3 hours
        self.GenerationSD = 3*60*60
        self.Transferrate = initDict[paramNamelist[11]]
    
    def update_storage(self):
        if self.Storagefailure == 1:
            excess_generation = self.Generation - self.Usage            
            self.Storage = self.Storage - self.Transfer + (self.Storageefficiency * excess_generation)
        else:
            if self.Generation > self.Usage:            
                self.Storage = self.Storage - self.Transfer
            else:
                self.Storage = self.Storage - self.Transfer - (self.Generation - self.Usage)  
        if self.Storage < 0:
            self.Storage = 0              
            self.Powercut = self.Powercut + 1
        if self.Storage > self.Maxstorage:
            self.Storage = self.Maxstorage
            
    def update_generation_failure(self):
        if self.Generationfailure == 1:
            gfra = np.random.binomial(1, (1-self.Generationfailurerate), 1)
            self.Generationfailure = gfra[0]
        else:
            if time > (self.Generationfailuetime+self.Generationrepairtime):
                self.Generationfailure == 1
    
    def update_storage_failure(self):
        if self.Storagefailure == 1:
            sfra = np.random.binomial(1, (1-self.Storagefailurerate), 1)
            self.Storagefailurerate = sfra[0]
        else:
            if time > (self.Storagefailuetime+self.Storagerepairtime):
                slef.Storagefailure == 1                
        
    def update_energy_generation(self,time):
        if self.Generationfailure == 1:
            self.Generation = self.Maxgeneration*(np.exp(-np.power((time%(24*60*60)) - self.Generationpeaktime, 2) / (2 * np.power(self.GenerationSD, 2))))
        else:
            self.Generation = 0
            
    def update_usage(self,time):
        self.Usage = self.Maxusage*(np.exp(-np.power((time%(24*60*60)) - self.Usagepeaktime, 2) / (2 * np.power(self.UsageSD, 2))))
    
    def update_transfer(self,dummysolarESagennts,solarNetwork):
        tr = 0
        if self.Name in solarNetwork.ndict:
            name1=self.Name
            for i in solarNetwork.ndict[self.Name]:            
                if dummysolarESagennts[name1].Storage < dummysolarESagennts[i].Storage:
                    tr=tr+1
                else:
                    if dummysolarESagennts[name1].Storage > dummysolarESagennts[i].Storage:
                        tr = tr-1        
        self.Transfer = tr
        
            
    def available_Energy(self):
        print("My available energy is " + str(self.Storage))

In [3]:
# Network class, to initiate as dictionary, add edge, add nodes, delete edge, delete node etc.,
class network:

    def __init__(self,ndict=None,):
        if ndict is None:
            ndict = {}
        self.ndict = ndict

    # get the lsit of nodes
    def getNodes(self):
        return list(self.ndict.keys())
    
    # get the list of edges
    def getEdges(self):
        return self.findedges()
    
    # finding the edge names
    def findedges(self):
        edgename = []
        for node in self.ndict:
            for nxtnode in self.ndict[node]:
                if (node, nxtnode) not in edgename:
                    edgename.append((node, nxtnode))
        return edgename

    # Add a new node as a key
    def addNode(self, node):
       if node not in self.ndict:
            self.ndict[node] = []
            
    # Add a new edge
    def addEdge(self, edge):
        (node1, node2) = tuple(edge)
        if node1 in self.ndict:
            if node2 not in self.ndict[node1]:
                self.ndict[node1].append(node2)
        else:
            self.ndict[node1] = [node2]
            
    # Delete a node
    def deleteNode(self, node):
       if node in self.ndict: del self.ndict[node]
            
    # Delete a edge
    def deleteEdge(self, edge):
        (node1, node2) = tuple(edge)
        if node1 in self.ndict:
            if node2 in self.ndict[node1]:
                del self.ndict[node1][self.ndict[node1].index(node2)]
    
    #get outward conections of a node
    def connections(self,node):
        if node in self.ndict:
            return self.ndict[node]
        else:
            return []
    def degree(self,node):
        if node in self.ndict:
            return len(self.ndict[node])
        else:
            return 0
    def countEdges(self):
        return sum([len(x) for x in self.ndict.values()])        

In [4]:
#This function will initiate the solarESagents, for given parameters (parmList and paramNamelist)
def initsolarES(noofsolarES,paramList,paramNamelist):    
    noofparam=len(paramList)
#Dictionary of initial parameters
    initList=list([])
    for i in range(0,noofsolarES):
        paramDict={}
        paramDict['name']=i
        for j in range(0,noofparam):
            paramDict[paramNamelist[j]]=paramList[j]
        initList.append(paramDict)
#For initiating solarES agents
    solarESagents=[]
    for k in range(0,noofsolarES):
        solarESagents.append(SolarES(paramNamelist,initList[k]))
    return(solarESagents)

In [5]:
# This function will generate the required solarNetwork (Barabási–Albert (BA) model).

def setup_BA_network(noofsolarES):
    network_elements = { 0 : [1],
                        1:[0]}
    n = network(network_elements)
    i=1
    while i < noofsolarES:
        i=i+1
        n.addNode(i)
        for j in range(i):
            nodedegree = n.degree(j)
            totaldegree = n.countEdges()
            attach_probab = nodedegree/totaldegree
            if np.random.uniform(0,1,1)[0]<attach_probab:
                n.addEdge((i,j))
                n.addEdge((j,i))
    return n

In [6]:
# This function will generate the required solarNetwork (Random model with given edge selection probability).
def setup_Random_network(noofsolarES,edge_selection_probability):   
    network_elements = { 0 : []}
    n = network(network_elements)
    for i in range(1,noofsolarES):
        n.addNode(i)
    for j in range(noofsolarES):
        for k in range(j+1,noofsolarES):
            if np.random.uniform(0,1,1)[0]<edge_selection_probability:
                n.addEdge((j,k))
                n.addEdge((k,j))
    return n

In [7]:
# This function will delete particular percentage of the edges from a given network
def connectionfailure(solarNetwork,percentagefailure):
    ed=solarNetwork.getEdges()
    k = len(ed) * percentagefailure // 2*100
    edges_todelete = random.sample(ed, k)
    for i in edges_todelete:
        solarNetwork.deleteEdge(i)
        solarNetwork.deleteEdge(i[::-1])

In [8]:
# This function will update the states of solarESagents given time and solarNetwork.
def updatesolarES(solarESagents,time,solarNetwork):    
    dummysolarESagennts=solarESagents    
    for i in solarESagents:                
        i.update_generation_failure()
        i.update_storage_failure()
        i.update_energy_generation(time)        
        i.update_usage(time)
        i.update_transfer(dummysolarESagennts,solarNetwork)
        i.update_storage()

In [9]:
# This defines the initiating parameters and initiates the Agent Based Simulations 
paramList=[3000,0.9,0,1000,0,5,0,0,0,0,14400,0,14400,14400]
paramNamelist=["maxenergy","storageefficiency","storage",'maxgeneration','generation','maxusage','usage','transfer','generationfailurerate','storagefailurerate','usage_SD','transferrate','generationrepairtime','storagerepairtime']
noofsolarES=50
edge_selection_probability=0.05
solarESagents=initsolarES(noofsolarES,paramList,paramNamelist)
solarNetwork=setup_Random_network(noofsolarES,edge_selection_probability)
totaltime = 24*60*60*5

In [10]:
# Simulations for choosen time period (here it is seconds)

for time in range(totaltime):
    updatesolarES(solarESagents,time,solarNetwork)